### Climate Models Use Different Calendars


Climate models can differ in their representation of the annual cycle.  While some climate models choose to follow the modern-day calendar with a Leap Day added every four years, other simulations choose instead to model every year has having exactly 365 days.   Still other climate models choose to make even stronger simplifying assumptions, such as months having equal duration.   For example, the HadGem Global Climate Model represents a year as having 12 months with 30 days each, for a calendar with 360 days a year.  

However, calendar differences can complicate the inter-comparison of climate model data.   What is the right way to compare data from a 365-day calendar against data from a 360-day calendar?   Generally, the answer depends on the specific details of the comparison.  But in cases where aggregate statistics are calculated, such as monthly temperature anomalies, a common approach is to *align* different calendars with each other.  Once this is done, it can be much easier to compare data across simulations. 




### Simple Calendar Conversion Using xarray
In some cases, calendars are similar enough that alignment is relatively straightforward.   For example, when aligning a calendar that takes into account Leap Years with a calendar that does not, one must either remove Feb.29 from the Leap year calendar, losing one day's worth of data every four years, or inserting a "missing day" of Feb. 29 into the "No Leap Year" calendar.  


### Simple Conversion Is Not Always Possible


In [ ]:
ds["time"] = ds.indexes["time"].to_datetimeindex()


## Custom Calendar Conversion:  When You Want Something Special


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import cftime
from datetime import date

In [ ]:
filePath = '/glade/collections/cdg/data/cordex/data/raw/NAM-22i/day/RegCM4/HadGEM2-ES/hist/tasmax/tasmax.hist.HadGEM2-ES.RegCM4.day.NAM-22i.raw.nc'
ds = xr.open_dataset(filePath)
ds

In [ ]:
## Create a small slice with the full time axis for local testing. 

#subset = ds.isel(lat=0, lon=0)
#subset
#subset.to_netcdf('/glade/u/home/bonnland/cal360_data.nc')

First, I define a custom function that returns an array of dates from the 365-day calendar, one date for each day of year on the 360-day calendar.  

In [ ]:
def get_datemap_360_to_noleap():
    ''' Return an array of dates mapping days from the 360-Day calendar to the No-Leap calendar. '''

    # Choose any year with 365 days. 
    dummy_year = 1999

    # These are the days of the year that will be missing on the time axis for each year. 
    # The goal is to spread missing dates out evenly over each year.
    #
    # Modify specific dates as desired. 
    missing_dates = [date(dummy_year, 1, 31),
                     date(dummy_year, 3, 31),
                     date(dummy_year, 5, 31),
                     date(dummy_year, 8, 31),
                     date(dummy_year, 10, 31)]
    
    day_one = date(dummy_year, 1, 1)
    missing_dates_indexes = [(day - day_one).days for day in missing_dates]    

    datemap_indexes = np.setdiff1d(np.arange(365), missing_dates_indexes)
    
    dates = pd.date_range(f'1/1/{dummy_year}', f'12/31/{dummy_year}')
    assert(len(dates) == 365)
    
    date_map = dates[datemap_indexes]
    assert(len(date_map) == 360)
    return date_map




Then I create a function that takes a particular date object from the 360-day calendar and returns the corresponding date from the 365-day calendar using the datemap generated in the previous function.

In [ ]:
def convert_to_noleap(cftime360, datemap):
    newdate = datemap[cftime360.dayofyr - 1]
    converted = cftime.DatetimeNoLeap(year=cftime360.year, month=newdate.month, day=newdate.day)
    return converted

In [ ]:
ds['time'] = [convert_to_noleap(t, datemap) for t in ds.time.values]
ds

Different calendars are chosen for different reasons, which may include ease of later analysis, or simply programming convenience. The [NA-CORDEX Climate Dataset](https://na-cordex.org/dataset-description.html) is one example of a climate dataset where different calendar assumptions are made in different files.   A summary of calendar differences can be found [here](https://na-cordex.org/time-ranges-calendars.html).